# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

**Проблемы обнаруженные при выводе результата метода info():**

1. В колонке "Трудовой стаж"(days_employed) значения указаны в формате вещественного типа. <u>**Необходимо перевести тип данных в целочисленный**</u>, так как в зачет в трудового стажа идут только полностью отработанные дни.
    
    
2. Количество записей (19351) в колонках "Трудовой стаж" (days_employed)  и "Ежемесячный доход"(total_income) не соответствует общему количеству записей (21525), а значит <u>**в колонках есть пропуски**.</u>
    
    Возможная причина появления пропусков кроется в значениях колонки "Тип занятости". Отсутствие трудового статуса и ежемесячного дохода может быть у неработающих людей, например : студенты или просто никогда не работавшие люди. И это гипотеза, а соответствует ли она действительности я проверю в процессе обработки пропусков.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
number_of_passes_in_days_employed = data['days_employed'].isna().sum()
number_of_passes_in_total_income = data['total_income'].isna().sum()
if number_of_passes_in_days_employed == number_of_passes_in_total_income:
    print(f'Количество пропусков в колонке "Трудовой стаж" равно количеству пропусков в колонке "Ежемесячный доход" - {number_of_passes_in_total_income}')
else:
    print(f"Количество пропусков в колонках не совпадает,\nКоличество пропусков в колонке Трудовой стаж до обработки - {number_of_passes_in_days_employed},\nКоличество пропусков в колонке Ежемесячный доход до обработки - {number_of_passes_in_total_income}\n")



Количество пропусков в колонке "Трудовой стаж" равно количеству пропусков в колонке "Ежемесячный доход" - 2174


In [3]:
people_with_nan = data[data['days_employed'].isna()]
people_with_nan.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Для начала сгруппирую людей без стажа и дохода в отдельный датафрейм. В срезе данных присутствую разные параметры. Значит зависимости отсутствия стажа и дохода от других параметров нет. Моя гипотеза не подтвердилась. Скорей всего пропуски допущены по технологическим причинам. Прежде чем обрабатывать пропуски, проверю значение в колонках в общем датафрейме на предмет ошибок значений.

In [4]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В колонках кроме пропусков присутствуют отрицательные значения. Понятие отрицательного стажа не существует, вероятно, что данные были перенесены в таблицу со знаком минус ошибочно. Возможно подобная ошибка есть и в других количественных значениях. Избавимся от отрицательного знака во всеx числовых значениях, и приведем все текстовые записи к единому регистру.

In [5]:
for column in data.columns:
    if data[column].dtypes == object:
        data[column] = data[column].str.lower()
    if (data[column].dtypes == 'int64') or (data[column].dtypes == 'float64'):
        data[column] = abs(data[column])
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу


При просмотре трудового стажа обнаружены записи, чьи значения превышают возраст человеческой жизни.
Возможно при заполнение была допущена ошибка в постановке разряда.
По статистике максимальный трудовой стаж составляет 20 лет или 7300 дней.
Это 4 разряда.
Найдем все записи, значение которых больше этого показателя, и переведем в допустимый разряд.

In [6]:
df_with_days_employed = data[data['days_employed'] > 20000]
for val in df_with_days_employed['days_employed']:
    if len(set(str(val))) > 4:
        new_val = val / 100
        data['days_employed'] = data['days_employed'].replace(val, new_val)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,3402.660720,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу


Проверим колонку с возрастом.

In [7]:
age_with_error = data[(data['dob_years'] <= 0) | (data['dob_years'] > 80)]
age_with_error['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

В колонку присутствуют нулевые значения,  всего их 101  запись.
Учитывая большую вариативность параметров, не вижу возможным корректно заполнить данный пропуск.
Исходя из отношения количества нулевых записей к общему, удаление таких данных не повлияет на дальнейшие расчеты.

In [8]:
data = data[data.dob_years != 0]

Заполним пропуски в колонках со стажем и доходом медианным значением.
Пропуски поделим по группам, основанным на типе занятости.
Каждой группе по типу занятости будет соответствовать свое медианное значение.

In [9]:
data['income_type_id'] = data['income_type'].map(
    {'студент': 0, 'безработный': 1, 'пенсионер': 2,
                    'в декрете':3, 'госслужащий': 4, 'сотрудник':5, 'компаньон':6, 'предприниматель':7} )
data.corr()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,income_type_id
children,1.000000,-0.068430,-0.179248,-0.013002,-0.091234,0.018239,0.017571,0.123174
days_employed,-0.068430,1.000000,0.382115,0.004821,-0.066990,-0.075188,-0.006106,-0.256665
dob_years,-0.179248,0.382115,1.000000,0.065190,-0.069103,-0.071919,-0.055998,-0.562510
education_id,-0.013002,0.004821,0.065190,1.000000,0.009357,0.052633,-0.179046,-0.090409
family_status_id,-0.091234,-0.066990,-0.069103,0.009357,1.000000,0.020950,-0.008970,0.008447
debt,0.018239,-0.075188,-0.071919,0.052633,0.020950,1.000000,-0.012421,0.035592
total_income,0.017571,-0.006106,-0.055998,-0.179046,-0.008970,-0.012421,1.000000,0.177075
income_type_id,0.123174,-0.256665,-0.562510,-0.090409,0.008447,0.035592,0.177075,1.000000


In [10]:
income_type = data[data['days_employed'].isna()]['income_type'].unique()
for val in income_type:
    median_for_total_income = data[data['income_type'] == val]['total_income'].median()
    median_for_days_employed = data[data['income_type'] == val]['days_employed'].median()
    data.loc[((data.income_type == val) & data.total_income.isna()), 'total_income'] = median_for_total_income
    data.loc[((data.income_type == val) & data.days_employed.isna()), 'days_employed'] = median_for_days_employed
    
print('Количество пропусков в колонке "Трудовой стаж" после обработки - ', (data['days_employed'].isna().sum()), '\n')
print('Количество пропусков в колонке "Ежемесячный доход" после обработки - ', (data['total_income'].isna().sum()), '\n')

Количество пропусков в колонке "Трудовой стаж" после обработки -  0 

Количество пропусков в колонке "Ежемесячный доход" после обработки -  0 



**Вывод**

**Обработка пропусков в колонках "Трудовой стаж" и "Ежемесячный доход".**

Пропуски в колонках могли быть связаны с технологической ошибкой.
Для заполнения пропусков в количественных переменных было выбранно медианное значение. Ее велична расчитана для каждого типа занятости отдельно. Среднее значение не использовалось, потому что могло бы некорректно охарактеризовать данные, если некоторые значения сильно выдавались бы среди большинства.

### Замена типа данных

In [11]:
data['days_employed'] = data['days_employed'].astype('int')
data['days_employed'].head()

0    8437
1    4024
2    5623
3    4124
4    3402
Name: days_employed, dtype: int64

**Вывод**

**Замена типа данных в колонке "Трудовой стаж".**

Вещественный тип данных был заменен на целочисленный, так как в зачет трудового стажа идут полностью отработанные года, месяца и дни. В нашем случае дни, поэтому все числа после запятой не будут иметь значения для расчета скоринга.

### Обработка дубликатов

In [12]:
print('Количетсво записей дубликатов - ',data.duplicated().sum())

Количетсво записей дубликатов -  71


In [13]:
data = data.drop_duplicates().reset_index(drop=True)
data.shape

(21353, 13)

**Вывод**

**Обработка дубликатов.**

Всего в дата фрейме была обнаружена 71 дублированная запись.
Дублирующие записи могли появится в результате ошибки ввода( например, при копировании из разных таблиц содержащих одинаковые записи, либо схожие параметры у некоторых людей).
Для обнаружения явных дубликатов использовался метод duplicated() + метод sum() для подсчета их количества.
Для удаления дубликатов я использовал метод drop_duplicated() и метод resete_index с параметром drop=True.
Такая связка позволит перезаписать датафрейм с обычным порядком индексов и без создания отдельной колонки со старыми индексами.

### Лемматизация

Для начала посмотрим на значения в клонке "Цель".

In [14]:
unique_purposes = data['purpose'].unique()
print(unique_purposes)
print('\nВсего уникальных значений - ',len(unique_purposes))

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']

Всего уникальных значений - 

Получено 38 уникальных значений.
На первый взгляд это немного, но при поверхностном осмотре видно, что некоторые цели повторяются.То есть одна цель выражена разными словосочетаниями-синонимами.
Посчитаем часто упоминающиеся цели, для этого лемматизирум каждое значение ( приведем к словарной форме), соеденим в общий список и с помощью метода Counter() посмотрим сколько раз повторяется каждое слово.

In [15]:
from pymystem3 import Mystem
from collections import Counter


list_with_all_purposes = []
m = Mystem()
for purpose in unique_purposes:
    lemmas = m.lemmatize(purpose)
    lemmas.pop(-1)
    list_with_all_purposes.extend(lemmas)
print(Counter(list_with_all_purposes))


Counter({' ': 59, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


В наборе присутствует слово и частота его повторения. Некоторыми способами можно удалить из списка с целями все глаголы , местоимения, в общем все кроме существительных( их я взял за основу как цель получения кредита). Но изящных и быстрых способов для такого процесса я не нашел. В связи с тем, что учебный массив небольшого размера, и действия по удалению "лишнего" займут много итераций, я в ручном режиме проанализировал и взял основные существительные-цели для составления функции. Она будет обрабатывать цель из колонки и вовзвращать отформатированное основное значение для новой колонки.

In [16]:
def formatting_the_purpose(val):
    lemmas = m.lemmatize(val)
    if ('строительство' in lemmas):
        return 'строительство'
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автотранспорт'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'ремонт' in lemmas:
        return 'ремонт'
    if 'образование' in lemmas:
        return 'образование'
    return 'цель не определена'


data['target'] = data['purpose'].apply(formatting_the_purpose)
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,target
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,5,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,5,автотранспорт
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,5,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,5,образование
4,0,3402,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,2,свадьба
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья,6,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем,6,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование,5,образование
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы,5,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи,5,недвижимость


**Вывод**

**Лемматизация.**

Для лемматизация использовал библиотеку pymystem3, она возвращает список лемматизированных слов для каждого значения колонки, так же библиотека работает и с несуществующими словами.
Предварительно получил список всех уникальных значений колонки, а затем лемматизировал эти значения, соеденил в общий список и посмотрел частоту повторения для исключения синонимов-дубликтов. С помощью функции в цикле лемматизировал каждое значение в колонке purpose и получил новое значение, которое добавил в дополнительную колонку методом aplly().

### Категоризация данных

Категоризировать  будущих заемщиков буду на основе их трудового стажа, работы, уровня дохода, возраста, семейного положения, наличия детей, образования, а так же возникали ли ранее проблемы с возвратом кредита.
Отдадим предпочтение клиентам состоящим в браке, но с минимальным количеством детей (в статьях на банковскую тематику пишут, что наличие детей снижает платежеспособность).
Так же перспективными заемщиками будут считаться люди с высшим и более образованием.
Сначала определим допустимый уровень дохода и трудовой стаж. Для их определения буду использовать метод median().

In [17]:
median_income = data['total_income'].median()
median_employed = data['days_employed'].median()
print('Медианный доход - ', median_income)
print('Медианный трудовой стаж - ', median_employed)

Медианный доход -  142594.39684740017
Медианный трудовой стаж -  1998.0


Теперь напишем функцию, которая будет принимать параметры, анализировать их и выдавать статус платежеспособности клинета.

In [18]:
def get_status(row):
    cnt = 0
    if ((row['days_employed'] >= median_employed) and
       (row['income_type'] in ['сотрудник', 'компаньон', 'госслужащий', 'предприниматель'])):
        cnt += 2
    if (21 <= row['dob_years'] <= 65) and (row['education'] in ['высшее', 'ученая степень' ]):
        cnt += 2
    if row['total_income'] >= median_income:
        cnt += 1
    if (row['family_status'] in ['женат/замужем', 'гражданский брак', 'не женат / не замужем']) and (row['children'] <= 1):
        cnt += 2
    if row['debt'] == 0:
        cnt += 1
    if row['income_type'] in ['студент', 'безработный', 'пенсионер']:
        cnt -= 1
    if row['debt'] == 1:
        cnt -= 2
    procent = (cnt / 8) * 100
    if procent >= 50:
        return 'платежеспособен'
    elif procent <= 50:
        return 'неплатежеспособен'

    

Применим функцию к строкам датафрейма с помощью метода apply() с параметром axis=1, что бы входными значениями для функции стали строки.

In [19]:
data['solvency_status'] = data.apply(get_status, axis=1)
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,target,solvency_status
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,5,недвижимость,платежеспособен
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,5,автотранспорт,неплатежеспособен
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,5,недвижимость,платежеспособен
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,5,образование,платежеспособен
4,0,3402,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,2,свадьба,неплатежеспособен
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья,6,недвижимость,платежеспособен
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем,6,недвижимость,платежеспособен
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование,5,образование,неплатежеспособен
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы,5,свадьба,платежеспособен
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи,5,недвижимость,платежеспособен


**Вывод**

**Категоризация.**

Проанализировав информацию из нескольких источников в интерете, я выбрал основные параметры и принцип категорирования.
На мой взгляд необходимо определить является ли клиент платежеспособным.
Для определения среднего уровня дохода и стажа использовал метод median().
Далее написал функцию , которая анализирует входные параметры и выдает итоговый статус платежеспособности.
И пременил метод apply с параметром axis=1 ( для перехода по всем строкам) ко всему датафрейму.
Итоговый вывод - колока с категориями платежеспособности клинетов на основе предаставленных данных.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
data.pivot_table(index=['children'], values='debt', aggfunc='mean').sort_values(by='debt', ascending=True).head(10)

,debt
children,
5,0.000000
0,0.075453
3,0.082317
1,0.091341
2,0.095145
4,0.097561
20,0.106667


In [21]:
df_with_5_children = data[data['children'] == 5]
print(df_with_5_children.shape)

(9, 15)


**Вывод**

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**
    
Группу клиентов с 5 детьми не будем брать в расчет, так как от нее всего 9 заявок c одиниковым значением в колонке "debt". Для анализа такое количетсво не подойдет. 
Минимальный процент задолженности по кредиту у группы без детей, макисмальный процент ( 10% шанс возникновения задержки платежа) у группы, где на каждого клиента по 20 детей.
Значения для всех групп находятся в одном диапозоне, считаю, что зависимости нет.

In [22]:
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

In [23]:
data[data['children'] == 20].count()

children            75
days_employed       75
dob_years           75
education           75
education_id        75
family_status       75
family_status_id    75
gender              75
income_type         75
debt                75
total_income        75
purpose             75
income_type_id      75
target              75
solvency_status     75
dtype: int64

В колонке "Количетсво детей" не сколько уникальных значений. В том числе, присутствует значение 20, которое слишком сильно "оторвано" от предыдущего значения. Возможно ноль дописан случайно. Приведем значение к корректному виду.

In [29]:
data['children'] = data['children'].replace(20,2)
data[data['children'] == 20].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
income_type_id      0
target              0
solvency_status     0
level_income        0
dtype: int64

In [31]:
data[data['children'] == 5].count()

children            9
days_employed       9
dob_years           9
education           9
education_id        9
family_status       9
family_status_id    9
gender              9
income_type         9
debt                9
total_income        9
purpose             9
income_type_id      9
target              9
solvency_status     9
level_income        9
dtype: int64

Клиентов у которых по 5 детей всего девять, считаю, что их можно исключить из выборки

In [33]:
data = data[data.children != 5]

In [34]:
data.pivot_table(index=['children'], values='debt', aggfunc='mean').sort_values(by='debt', ascending=True).head(10)

,debt
children,
0,0.075453
3,0.082317
1,0.091341
2,0.095553
4,0.097561


Минимальный процент задолженности по кредиту у группы без детей, макисмальный процент ( 10% шанс возникновения задержки платежа) у группы, где на каждого клиента по 4 детей.
Более высокой платежеспособностью обладают клиенты без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
data.pivot_table(index=['family_status'], values='debt', aggfunc='mean').sort_values(by='debt', ascending=True).head()

,debt
family_status,
вдовец / вдова,0.064990
в разводе,0.071730
женат / замужем,0.075427
гражданский брак,0.093462
не женат / не замужем,0.097709


**Вывод**

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**
    
При большом количестве потенциальных кредиторов это будет ощутимая разница. Тогда минимальных процент не возврата у группы "вдова/вдовец", а макисмальный у группы "не женат / не замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для начала определи минимальный и максимальный доход, затем возьмем медианное значение и шаг в стороны по 50000 для определения диапазона со среденим доходом.После напишем функцияю для метода apply(), которая будет принимать на вход значение из стоблца "total_income" и возвращать в новую колонку level_income значение доходности клиента.

In [26]:
min_income = data['total_income'].min()
low_bound = median_income - 50000
high_bound = median_income + 50000
max_income = data['total_income'].max()

def get_income_level(income):
    if min_income < income < low_bound:
        return 'низкий доход'
    elif low_bound < income < high_bound:
        return 'средний доход'
    elif high_bound < income < max_income:
        return 'высокий доход'

data['level_income'] = data['total_income'].apply(get_income_level)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,target,solvency_status,level_income
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,5,недвижимость,платежеспособен,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,5,автотранспорт,неплатежеспособен,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,5,недвижимость,платежеспособен,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,5,образование,платежеспособен,высокий доход
4,0,3402,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,2,свадьба,неплатежеспособен,средний доход


In [27]:
data.pivot_table(index=['level_income'], values='debt', aggfunc='mean').sort_values(by='debt', ascending=True).head()

,debt
level_income,
высокий доход,0.071880
низкий доход,0.079678
средний доход,0.085735


**Вывод**

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**
    
У клиентов с низким уровнем дохода процент шанса задержки платежа выше чем у остальных групп, минимальный у клиентов с высоким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
data.pivot_table(index=['target'], values='debt', aggfunc='mean').sort_values(by='debt', ascending=True).head()

,debt
target,
недвижимость,0.071421
строительство,0.076882
свадьба,0.079654
образование,0.092616
автотранспорт,0.093371


**Вывод**

**Как разные цели кредита влияют на его возврат в срок?**
    
Минимальный процент невозврата у клиентов, которые берут кредит на недвижимость, а максимальный у "автотранспорт".

## Шаг 4. Общий вывод

**Общий вывод**
    
Исходя из полученный результатов, я сделал вывод, что предпочтительнее выбирать будущих клиентов из числа потенциальных кредиторов:
    
 - с высоким доходом;
    
    
 - без детей или с детьми , но не больше 1 (наличие > 1 ребенка снижает платежеспособность);
    
    
 - состоящие в браке или разведенные, так же вдовы / вдовцы;
    
    
 - с целями имеющими долгосрочные обязательства, например, кредит на покупку недвижимости или строительства;